In [27]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import ParameterSampler, RandomizedSearchCV, cross_val_score

# Load data and transform

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


In [7]:
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
# imshow(torchvision.utils.make_grid(images))

# Define Convolutional Model

In [8]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Training loop

In [10]:
def train_net(net, trainloader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 300 == 0:    # print every 2000 mini-batches
                print(f'[{epoch+1}, {i:5d}] loss: {running_loss / 300:.3f}')
                running_loss = 0.0

    print('Finished Training')

train_net(net, trainloader, criterion, optimizer, num_epochs=10)


[1,     0] loss: 0.008
[1,   300] loss: 2.303
[1,   600] loss: 2.298
[1,   900] loss: 2.279
[1,  1200] loss: 2.193
[1,  1500] loss: 2.082
[2,     0] loss: 0.007
[2,   300] loss: 1.956
[2,   600] loss: 1.900
[2,   900] loss: 1.833
[2,  1200] loss: 1.790
[2,  1500] loss: 1.711
[3,     0] loss: 0.005
[3,   300] loss: 1.643
[3,   600] loss: 1.616
[3,   900] loss: 1.602
[3,  1200] loss: 1.555
[3,  1500] loss: 1.541
[4,     0] loss: 0.005
[4,   300] loss: 1.490
[4,   600] loss: 1.475
[4,   900] loss: 1.470
[4,  1200] loss: 1.434
[4,  1500] loss: 1.422
[5,     0] loss: 0.004
[5,   300] loss: 1.410
[5,   600] loss: 1.392
[5,   900] loss: 1.353
[5,  1200] loss: 1.357
[5,  1500] loss: 1.344
[6,     0] loss: 0.005
[6,   300] loss: 1.303
[6,   600] loss: 1.307
[6,   900] loss: 1.290
[6,  1200] loss: 1.310
[6,  1500] loss: 1.261
[7,     0] loss: 0.003
[7,   300] loss: 1.247
[7,   600] loss: 1.238
[7,   900] loss: 1.239
[7,  1200] loss: 1.237
[7,  1500] loss: 1.211
[8,     0] loss: 0.004
[8,   300] 

In [ ]:
# Save model
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

# Test Loop

In [21]:
def test_net(net, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the 10000 test images: {accuracy:.1f} %')
    return accuracy

# correct = 0
# total = 0
# # since we're not training, we don't need to calculate the gradients for our outputs
# with torch.no_grad():
#     # for data in trainloader: # Test on the training set
#     for data in testloader: # Test on the test set (This is what we want to do)
#         images, labels = data
#         # calculate outputs by running images through the network
#         outputs = net(images)
#         # the class with the highest energy is what we choose as prediction
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

test_net(net, testloader)

Accuracy of the network on the 10000 test images: 58.8 %


58.84

# Define domain of parameters to be optimized

In [29]:
## define the domain of the considered parameters
n_estimators = tuple(np.arange(1,101,1, dtype=int))
# print(n_estimators)
max_depth = tuple(np.arange(10,110,10, dtype=int))
# max_features = ('log2', 'sqrt', None)
max_features = (0, 1)
# criterion = ('gini', 'entropy')
criterion = (0, 1)


# define the dictionary for GPyOpt
domain_random_sample = {
        "n_epochs": range(1, 101), 
        "n_layers": range(2,5),
        "": range(10, 60, 5),
        "max_features": ['sqrt', 'log2']
        }

domain = [{'name': 'n_estimators', 'type': 'discrete', 'domain':n_estimators},
          {'name': 'max_depth', 'type': 'discrete', 'domain': max_depth},
          {'name': 'max_features', 'type': 'categorical', 'domain': max_features},
          {'name': 'criterion', 'type': 'categorical', 'domain': criterion}]

In [31]:
param_list = list(ParameterSampler(domain_random_sample, n_iter=5, random_state=32))
param_list

[{'n_estimators': 76,
  'max_features': 'log2',
  'max_depth': 25,
  'criterion': 'entropy'},
 {'n_estimators': 80,
  'max_features': 'log2',
  'max_depth': 45,
  'criterion': 'gini'},
 {'n_estimators': 74,
  'max_features': 'log2',
  'max_depth': 55,
  'criterion': 'entropy'},
 {'n_estimators': 35,
  'max_features': 'log2',
  'max_depth': 40,
  'criterion': 'gini'},
 {'n_estimators': 41,
  'max_features': 'log2',
  'max_depth': 30,
  'criterion': 'entropy'}]

In [17]:


def objective_function(x):
    net = Net()
    train_net(net, trainloader, criterion, optimizer, num_epochs=1)
    accuracy = test_net(net, testloader)
    return accuracy


acc = objective_function(None) # Warm up
    

[1,     0] loss: 0.008
[1,   300] loss: 2.306
[1,   600] loss: 2.305
[1,   900] loss: 2.304
[1,  1200] loss: 2.305
[1,  1500] loss: 2.305
Finished Training
Accuracy of the network on the 10000 test images: 10 %


In [19]:
print(acc)

None
